In [15]:
import os
import sys
import numpy as np
import torch

sys.path.append(os.path.abspath('..'))

from env.env import BinPacking3DEnv
from models.model import BinPackingModel

In [16]:
# Create an environment and dummy input to test the model
env = BinPacking3DEnv(
	bin_size=(5, 5, 5),
	items=[(2, 3, 1), (2, 2, 3), (1, 1, 2), (3, 2, 2)],
	buffer_size=2,
	num_rotations=2,
)

obervation, _ = env.reset()

In [17]:
# EMS list
ems_list = obervation['ems_list']

ems_list.shape # [?, 6]

(1, 6)

In [18]:
# Buffer
buffer = obervation['buffer']

buffer.shape # [buffer_size, 3]

(2, 3)

In [19]:
# Action mask
action_mask = obervation['action_mask']

action_mask.shape # [W, L, num_rotations, buffer_size]

(5, 5, 2, 2)

In [2]:
# Create a model 
model = BinPackingModel(
	d_action=env.W * env.L * env.num_rotations * env.buffer_size,
)

In [3]:
model

BinPackingModel(
  (transformer): BinPackingTransformer(
    (ems_list_embedding): CombinedEmbedding(
      (conv): Conv1d(6, 128, kernel_size=(1,), stride=(1,))
    )
    (buffer_embedding): CombinedEmbedding(
      (linear): Linear(in_features=3, out_features=128, bias=True)
    )
    (transformer_blocks): ModuleList(
      (0-2): 3 x TransformerBlock(
        (self_attn_ems_list): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (self_attn_buffer): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (norm1_ems_list): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm1_buffer): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp_ems_list): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, ou

In [4]:
total_params = sum(p.numel() for p in model.parameters())

total_params

2989826

In [9]:
# Expand the dimensions of the input to match the model's input shape
ems_list_np = np.expand_dims(ems_list, axis=0)  # [1, num_ems, 6]
buffer_np = np.expand_dims(buffer, axis=0)      # [1, buffer_size, 3]
action_mask_np = np.expand_dims(action_mask, axis=0)  # [batch_size=1, W, L, num_rotations, buffer_size]

ems_list_np.shape, buffer_np.shape, action_mask_np.shape

((1, 1, 6), (1, 2, 3), (1, 5, 5, 2, 2))

In [10]:
# Convert the numpy arrays to tensors
ems_list_tensor = torch.tensor(ems_list_np, dtype=torch.float32)
buffer_tensor = torch.tensor(buffer_np, dtype=torch.float32)
action_mask_tensor = torch.tensor(action_mask_np, dtype=torch.float32)


ems_list_tensor.shape, buffer_tensor.shape, action_mask_tensor.shape

(torch.Size([1, 1, 6]), torch.Size([1, 2, 3]), torch.Size([1, 5, 5, 2, 2]))

In [14]:
# action_mask_tensor must be [batch_size, W * L * num_rotations * buffer_size]
action_mask_tensor = action_mask_tensor.view(1, -1)

action_mask_tensor.shape

torch.Size([1, 100])

In [11]:
# Choose the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [12]:
# Move the model and input tensors to the device
model.to(device)

ems_list_tensor = ems_list_tensor.to(device)
buffer_tensor = buffer_tensor.to(device)
action_mask_tensor = action_mask_tensor.to(device)

In [13]:
# Set the model to evaluation mode
model.eval()

with torch.no_grad():
	value, policy = model(ems_list_tensor, buffer_tensor, action_mask_tensor)

value.shape, policy.shape

RuntimeError: The size of tensor a (128) must match the size of tensor b (2) at non-singleton dimension 4